In [53]:
import pandas as pd 
import numpy as np

test_data = pd.read_csv('/Users/estene/Documents/GitHub/collections-as-data-notebooks/data/kaplan-test-data.csv')
new_data = pd.read_csv('/Users/estene/Documents/GitHub/kaplan-collection/data/kaplan_colenda.csv')

revised_data = pd.merge(test_data, new_data, left_on='unique_identifier', right_on='ark', how='outer')
revised_data.to_csv("revised_colenda_data.csv", index=False)

/var/folders/kl/0dwxwfpd25xdhw1l6w4r3pbr0000gs/T/ipykernel_40257/1032123041.py:4: DtypeWarning: Columns (2,7,10,16,23,24,25,26,27,28,29,30,31,32,33,44,45,46,47,48,49,53,54,55,56,57,58,61,62,63,64,65,66,67,68,69,70,72,76,77,78,79,80,82,84,85,86,95) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('/Users/estene/Documents/GitHub/collections-as-data-notebooks/data/kaplan-test-data.csv')


In [54]:
trade_cards = pd.read_csv('/Users/estene/Documents/GitHub/collections-as-data-notebooks/testing notebooks/trade_cards_coordinates.csv')
revised_trade_cards = pd.merge(trade_cards, test_data, left_on="ark", right_on='unique_identifier', how="left")
revised_trade_cards

,ï»¿title,date,description,language,type,personal_name,geographic_subject,call_number,collection,identifiers,...,metadata.subject[2],metadata.subject[3],metadata.subject[4],metadata.subject[5],metadata.subject[6],metadata.subject[7],metadata.subject[8],metadata.subject[9],metadata.title[1],unique_identifier
0,"Trade card; Harry Cohen; Philadelphia, Pennsyl...",unknown,Harry Cohen: Dealer in Hardware and Sporting G...,English,Trade cards,NaN,United States -- Pennsylvania -- Philadelphia;...,NaN,Arnold and Deanne Kaplan Collection of Modern ...,2006.12.26.04988; {D980421B-D4B8-4F7A-A47C-70A...,...,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,"Trade card; Harry Cohen; Philadelphia, Pennsyl...",ark:/81431/p3fn10r57
1,"Trade card; O. V. Handy & Bros.; Philadelphia,...",1916,"O. V. Handy & Bros. Philadelphia, PA; 312 Sout...",English,Trade cards,NaN,United States -- Pennsylvania -- Philadelphia;...,NaN,Arnold and Deanne Kaplan Collection of Modern ...,2006.12.26.05257; {F0522573-795A-4A9C-BD2D-B85...,...,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,"Trade card; O. V. Handy & Bros.; Philadelphia,...",ark:/81431/p3gf0mv9t
2,"Trade card; Standard Hosiery Co., I. Berlizhei...",unknown,D. T. Berlizheimer: Standard Hosiery Co.; 1310...,English,Trade cards,NaN,United States -- Pennsylvania -- Philadelphia;...,NaN,Arnold and Deanne Kaplan Collection of Modern ...,2006.12.26.04989; {75A18626-30EC-4D98-9FE6-39F...,...,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,"Trade card; Standard Hosiery Co., I. Berlizhei...",ark:/81431/p3q52fc3r
3,"Trade card; S. Jacobson; Philadelphia, Pennsyl...",unknown,"S. Jacobson: Wholesale and Retail Diamonds, Wa...",English,Trade cards,NaN,United States -- Pennsylvania -- Philadelphia;...,NaN,Arnold and Deanne Kaplan Collection of Modern ...,2006.12.26.04987; {5139AAFF-3631-4F66-8297-7F5...,...,Clock and watch industry,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,"Trade card; S. Jacobson; Philadelphia, Pennsyl...",ark:/81431/p3zg6g65p
4,"Trade card; H. Levi; Philadelphia, Pennsylvani...",unknown,H. Levi: Money Loan Office; Southeast Corner o...,English,Trade cards,NaN,United States -- Pennsylvania -- Philadelphia;...,NaN,Arnold and Deanne Kaplan Collection of Early A...,2006.12.26.08288; {524598C4-1815-442E-A33E-FAF...,...,Jewish capitalists and financiers,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,"Trade card; H. Levi; Philadelphia, Pennsylvani...",ark:/81431/p3000026s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,Trade card; Friedberger & Strouse; Philadelphi...,1877,"Friedberger & Strouse: Ribbons, Laces, Velvets...",English,Trade cards,NaN,United States -- Pennsylvania -- Philadelphia;...,NaN,Arnold and Deanne Kaplan Collection of Early A...,2006.12.26.07835; {FA7C4CD6-B4B2-46E3-9B53-598...,...,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,Trade card; Friedberger & Strouse; Philadelphi...,ark:/81431/p3mg7fv52
479,"Trade card; Snyder, Harris, Bassett & Co.; Phi...",unknown,"Snyder, Harris, Bassett & Co.; 525 Market Stre...",English,Trade cards,NaN,United States -- Pennsylvania -- Philadelphia;...,NaN,Arnold and Deanne Kaplan Collection of Early A...,2006.12.26.04827; {5C62BAE8-6933-4E0B-B8C3-006...,...,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,"Trade card; Snyder, Harris, Bassett & Co.; Phi...",ark:/81431/p3pz51k8n
480,Trade card; N. Snellenburg & Co.; Philadelphia...,unknown,"N. Snellenburg & Co.: Clothiers; Fifth, South ...",English,Trade cards,NaN,United States -- Pennsylvania -- Philadelphia;...,NaN,Arnold and Deanne Kaplan Collection of Early A...,2007.01.14.00036; {608AAB54-D9AF-458E-BC03-D15...,...,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,Trade card; N. Snellenburg & Co.; Philadelphia...,ark:/81431/p3q23r03w
481,Trade card; Friedberger & Strouse; Philadelphi...,unknown,"Friedberger & Strouse: Handk

In [55]:
def tidy_concat(df, column_starts_with, sep="|"):
    """
    Params
    ------
    df : pandas.DataFrame
        dataframe with the columns to split and expand
    column_starts_with : str
        the string at the beginning of the column(s) to split
    sep : str
        the string used to split the column's values

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    list_of_columns = df.columns.to_list()
    columns_to_concat = [x for x in list_of_columns if x.startswith(column_starts_with)]
    df[column_starts_with] = df[columns_to_concat[0]]
    for column in columns_to_concat[1:]:
        df[column_starts_with] = df[column_starts_with].astype(str) + sep + df[column].astype(str)
    new_df = tidy_split(df, column_starts_with, sep='|')
    new_df = new_df.drop(columns_to_concat, axis=1)
    return new_df

In [56]:
def tidy_split(df, column, sep='|', keep=False):
    """
    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [57]:
df_places = tidy_concat(revised_trade_cards, 'metadata.subject', sep='|')
df_places.dropna(subset=['metadata.subject'])
#df_places = df_places[df_places['metadata.subject'!=np.nan]]
df_places.to_csv("/Users/estene/Downloads/trade_cards_with_subjects.csv")

In [58]:
top_types = df_places['metadata.subject'].value_counts()#[:10].index.to_list()
top_types

nan                                  3994
Trade cards (advertising)             480
Jewish merchants                      469
Clothing trade                        217
Dress accessories                     102
Decorative arts                        87
Paper products industry                58
Artists' studios                       56
Advertisements                         45
Dry-goods                              44
Clock and watch industry               27
Jewelry trade                          24
House furnishings                      20
Jewelry stores                         18
Amusements                             15
Furniture industry and trade           13
Prejudices                             12
Caricatures                            12
Food industry and trade                11
Umbrellas                              10
International trade                    10
Animal industry                        10
Specialty stores                        9
Drugstores                        

In [59]:
items_care_about = ["Clothing trade","Dress accessories","Decorative arts","Paper products industry","Artists' studios","Advertisements","Dry-goods","Clock and watch industry","Jewelry trade","House furnishings","Jewelry stores","Amusements","Furniture industry and trade","Prejudices","Caricatures","Food industry and trade","Umbrellas","International trade","Animal industry","Specialty stores","Drugstores","Jewish pharmacists","Containers","Art","Printing industry","Tobacco industry","Rabbis","Loans","Interior decoration","Textile industry","Publishers and publishing","Book industries and trade","General stores","Hotel keepers","Construction industry","Jewish florists","Brokers","Boardinghouses","Artists' materials","Infants' supplies","Jewish printers","Jewish lawyers","Secondhand trade","Vehicles","Broom and brush industry","Discount houses (Retail trade)","Jewish artists","Jewish capitalists and financiers","Hygiene products"]
new_df = df_places[df_places['metadata.subject'].isin(items_care_about)]
#new_df.shape
new_df.drop_duplicates(subset ="unique_identifier",keep='first', inplace = True)
new_df.drop_duplicates(subset ="addresses",keep='first', inplace = True)
new_df
new_df.to_csv("/Users/estene/Downloads/trade_cards_with_subjects.csv")

/var/folders/kl/0dwxwfpd25xdhw1l6w4r3pbr0000gs/T/ipykernel_40257/1544672389.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop_duplicates(subset ="unique_identifier",keep='first', inplace = True)
/var/folders/kl/0dwxwfpd25xdhw1l6w4r3pbr0000gs/T/ipykernel_40257/1544672389.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop_duplicates(subset ="addresses",keep='first', inplace = True)
